In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score


In [ ]:
df = pd.read_csv('../../../datasets/parte2/treino/dataset_prepared.csv', na_filter= False)

In [ ]:
lb_make = LabelEncoder()
df['injection'] = lb_make.fit_transform(df['injection'])

X = df.drop('injection', axis=1)
y = df[['injection']]

### Normalization

In [ ]:
scaler_X = MinMaxScaler(feature_range=(0, 1)).fit(X)
scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(y)
X = pd.DataFrame(scaler_X.transform(X[X.columns]), columns=X.columns)
#y = pd.DataFrame(scaler_y.transform(y[y.columns]), columns=y.columns)

In [ ]:
df.head

#### Train Test Split

Now let's split the data into a training set and a testing set. We will train out model on the training set and then use the test set to evaluate the model.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2023, stratify=y)

num_classes = 5
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
def build_model(activation='relu', learning_rate=0.005, dropout_rate=0.1):
    model = Sequential()
    model.add(Dense(64, input_dim=X.shape[1], activation=activation ))
    model.add(Dropout(dropout_rate))  # Adiciona Dropout após a primeira camada densa
    model.add(Dense(32, activation=activation))
    model.add(Dropout(dropout_rate))  # Adiciona Dropout após a segunda camada densa
    model.add(Dense(5, activation='softmax')) # output 
    #Compile the model
    model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy'])
    return model 

In [ ]:
model = build_model()
model.summary()

In [ ]:
param_grid = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad'] 
} 

In [ ]:
#kf = KFold(n_splits=5, shuffle=True, random_state=2023)

In [ ]:
model = KerasClassifier(model=build_model, batch_size=32, validation_split=0.2, epochs=100)

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=15, scoring='accuracy', refit=True, verbose=2, n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
best_mlp_model = grid_search.best_estimator_
print(best_mlp_model)

In [ ]:
best_mlp_model.fit(X_train, y_train, epochs=100, validation_data = (X_test, y_test), verbose = 1)

#### Learning Curves

In [ ]:
plt.plot(best_mlp_model.history_['loss'])
plt.plot(best_mlp_model.history_['val_loss'])
plt.title('Model Performance')
plt.ylabel('Loss values')
plt.xlabel('Epochs')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
predictions = best_mlp_model.predict(X_test)

In [ ]:
accuracy_score(y_test, predictions)

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
#df_predicoes = pd.DataFrame({'Result': predictions})
#df_predicoes['RowId'] = range(1, len(predictions) + 1)
#df_predicoes = df_predicoes[['RowId', 'Result']]

#df_predicoes.to_csv('../../../datasets/parte2/teste/neuralNetwork.csv', index=False)